# Google Spreadsheet Interface

## Open Data Science Initiative

### 14th August 2014 Neil Lawrence

As part of the distribution of information to area chairs in NIPS 2014 we used Google docs to provide summary spreadsheets. Review information is processed in `pandas` so this rough interface is designed to allow uploading and downloading of information from google spreadsheets as pandas data frames.

In [1]:
import pods.google
import pandas as pd

Next you'll need to make use of two gmail user IDs for this example.


In [2]:
user1 = 'user1@gmail.com'
user2 = 'user2@gmail.com'

First let's create a simple data frame for placing in a spreadsheet.

In [3]:
d = {'one' : pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
    'two' : pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])} 

df = pd.DataFrame(d)

In [4]:
df

,one,two
a,1,1
b,2,2
c,3,3
d,NaN,4


### Creating a Google Sheet
Now we create the google document and write our data frame, `df`, to the spreadsheet.

In [5]:
sheet = pods.google.sheet()
sheet.write(df)

The spreadsheet is uniquely defined by a key, which can be recovered from the spreadsheet as follows:

In [ ]:
print sheet._key

The spreadsheet can be shared (by default with 'edit' access) to other gmail users.

In [7]:
sheet.share([user1])

To share with 'view' access you pass an extra argument, `share_type='reader'`. You can also send a notification (by default there is no mail sent).

In [8]:
sheet.share([user2],share_type='reader',send_notifications=True)

The spreadsheet's title can also be renamed.

### Loading an Existing Sheet

Now, for demonstration pruposes, here's the key of 'one we made earlier'. You can open an existing spreadsheet by providing the key as follows

In [9]:
sheet = pods.google.sheet(spreadsheet_key='1kWBLPFhGt8PbbVGq_cSxfUAsBUjiA7OS3T_R7OHk0hk')

For this google doc, now it's created we've published it to the web. Quoting from [the developer's site](https://developers.google.com/google-apps/spreadsheets/)

"Publishing a spreadsheet to the web can only be done from the Google Spreadsheets user interface. To start publishing a spreadsheet to the web, select File > Publish to the web from the Spreadsheets user interface, and then click the Start Publishing button."

Once the spreadsheet is published you can have people viewing it publicly as follows.

In [10]:
sheet.share([user2])

Or you can modify the access rights they have to the document.

In [11]:
acl_feed = sheet.share_modify(user2, 'writer')

Or you can remove them from the sharing list completely.

In [12]:
sheet.share_delete(user2)

To see who the document is shared with you can use the `share_list()` method.

In [ ]:
sheet.share_list()

### Displaying the Sheet

You can display the sheet in IPython, this action downloads the sheet and shows the associated `pandas` dataframe.

In [14]:
from IPython.display import display
display(sheet)


,one,two
d,NaN,4
a,1,1
b,2,2


Or if the spreadsheet is published you can set `sheet.published = True` and get a representation of the spreadsheet embedded in the notebook.

In [15]:
sheet.published = True
display(sheet)

<pods.google.sheet instance at 0x10d9fe3f8>

## Reading and Modifying the Google Sheet

To read the data frame back from the spreadsheet you simply use the `read` method.

In [16]:
df2 = sheet.read()

We can now update the spreadsheet, by modifying the data frame, and then requesting an update. The update command looks only for entries that have changed.

In [17]:
df2['one']['a'] ='cat'
sheet.update(df2)

You should see the spreadsheet has changed. You can also delete rows.

In [18]:
sheet.update(df2.drop('a'))
display(sheet)

Warning deleting row indexed by 'a' from 3 currently! Not comprehensively tested. Best guess is that row to delete is 1 in Google spreadsheet https://docs.google.com/spreadsheets/d/1kWBLPFhGt8PbbVGq_cSxfUAsBUjiA7OS3T_R7OHk0hk/
Delete row (Y/N)?Y
Delete row  1


<pods.google.sheet instance at 0x10d9fe3f8>

Or add them in again. Here we update back to the original data frame (`df`). Note when you look at the google doc that the row is added at the end of the spreadsheet. This is so that any sorting of other rows in the spreadsheet is preserved.

In [19]:
sheet.update(df)

Let's open the sheet again with a new interface.

In [20]:
sheet2 = pods.google.sheet(spreadsheet_key=sheet._key)
df4 = sheet2.read()
print df4

   one  two
d  NaN    4
b    2    2
a    1    1
c    3    3


Note that because we deleted the row indexed by `'b'` and then updated the spreadsheet, the row of `b` was added back at the end of the spreadsheet.

In [21]:
sheet.update(df4.drop('c'))

Warning deleting row indexed by 'c' from 5 currently! Not comprehensively tested. Best guess is that row to delete is 3 in Google spreadsheet https://docs.google.com/spreadsheets/d/1kWBLPFhGt8PbbVGq_cSxfUAsBUjiA7OS3T_R7OHk0hk/
Delete row (Y/N)?Y
Delete row  3


In [22]:
display(sheet2)

,one,two
d,NaN,4
b,2,2
a,1,1
